# Επεξεργασία δεδομένων & δημιουργία Sliding Windows

In [8]:
# CNN για εκτίμηση οδηγικής ασφάλειας με Grid Search & hold-out prediction

# Εισαγωγή απαραίτητων βιβλιοθηκών
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Βιβλιοθήκες PyTorch για τη δημιουργία και εκπαίδευση του νευρωνικού δικτύου
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Βιβλιοθήκες Scikit-learn για διαχωρισμό δεδομένων και αξιολόγηση
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder # Για κωδικοποίηση ετικετών (π.χ. 'safe' σε 0, 'risky' σε 1)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from itertools import product # Για τη δημιουργία συνδυασμών παραμέτρων για το Grid Search

# === 1. Κλάση Επεξεργασίας Δεδομένων Επιτάχυνσης ===
# Αυτή η κλάση είναι υπεύθυνη για την ανάγνωση, προ επεξεργασία και δημιουργία sliding windows (παραθύρων)
# από τα δεδομένα επιτάχυνσης και τις αντίστοιχες ετικέτες.
class DrivingDataProcessor:
    # Ο κατασκευαστής της κλάσης
    def __init__(self, csv_file):
        # Έλεγχος αν υπάρχει το αρχείο δεδομένων επιτάχυνσης
        if not os.path.exists(csv_file):
            raise FileNotFoundError(f"Δεν βρέθηκε το αρχείο δεδομένων: {csv_file}")
        self.df = pd.read_csv(csv_file) # Ανάγνωση του αρχείου σε DataFrame

        # Έλεγχος αν υπάρχουν οι απαραίτητες στήλες στο αρχείο
        required_columns = [
            'Acceleration_X_Lateral',
            'Acceleration_Y_Longitudinal',
            'Acceleration_Z_Vertical',
            'Global_Time_Seconds'
        ]
        if not all(col in self.df.columns for col in required_columns):
            raise ValueError(f"Το αρχείο πρέπει να περιέχει τις στήλες: {required_columns}")

        # Αρχικοποίηση LabelEncoder για την κωδικοποίηση των ετικετών.
        # Ελέγχουμε αν υπάρχει ήδη η στήλη ετικετών στο αρχείο δεδομένων.
        # Αν όχι, τη δημιουργούμε με κλάσεις [0, 1] (ασφαλές, επικίνδυνο).
        self.label_encoder = LabelEncoder()
        if 'Driving_Safety_Assessment' in self.df.columns:
            self.df['label_encoded'] = self.label_encoder.fit_transform(
                self.df['Driving_Safety_Assessment']
            )
        else:
            self.label_encoder.fit([0, 1])

    # Συνάρτηση για τη δημιουργία sliding windows (παραθύρων) από τα δεδομένα
    # και τη συσχέτισή τους με τις ετικέτες από ένα ξεχωριστό αρχείο.
    def create_sequences_with_labels(self, window_size, label_file):
        # Έλεγχος αν υπάρχει το αρχείο ετικετών
        if not os.path.exists(label_file):
            raise FileNotFoundError(f"Δεν βρέθηκε το αρχείο ετικετών: {label_file}")

        # Ανάγνωση των ετικετών από το αρχείο (υποθέτουμε ότι είναι σε μία στήλη χωρίς header)
        labels = pd.read_csv(label_file, header=None).values.squeeze()
        # Ταξινόμηση του DataFrame με βάση το χρόνο για να εξασφαλίσουμε τη χρονική σειρά
        sorted_df = self.df.sort_values('Global_Time_Seconds')
        # Επιλογή των στηλών επιτάχυνσης
        acc_data = sorted_df[['Acceleration_X_Lateral',
                              'Acceleration_Y_Longitudinal',
                              'Acceleration_Z_Vertical']].values

        # Έλεγχος αν υπάρχουν αρκετά δεδομένα για να δημιουργήσουμε τα παράθυρα
        total_needed = window_size * len(labels)
        if len(acc_data) < total_needed:
            raise ValueError("Δεν υπάρχουν αρκετά δεδομένα για όλα τα παράθυρα.")

        # Δημιουργία των ακολουθιών (παραθύρων)
        sequences = []
        for i in range(len(labels)):
            start = i * window_size # Υπολογισμός της αρχής του παραθύρου
            end = start + window_size # Υπολογισμός του τέλους του παραθύρου
            sequences.append(acc_data[start:end]) # Προσθήκη του παραθύρου στη λίστα

        # Επιστροφή των ακολουθιών και των ετικετών (ως λίστα)
        return sequences, labels.tolist()


# Φόρτωση των δεδομένων σε batches & Προσθήκη θορύβου Jitter Guassian για Augmentation

In [9]:
# === 2. Κλάση Dataset για PyTorch ===
# Φόρτωση των δεδομένων σε batches για την εκπαίδευση του μοντέλου PyTorch.
class SlidingWindowDataset(Dataset):
    # Ο κατασκευαστής της κλάσης
    def __init__(self, sequences, labels, augment=False):
        self.sequences = np.array(sequences) # Μετατροπή των ακολουθιών σε NumPy array
        self.labels = labels # Αποθήκευση των ετικετών
        self.augment = augment # data augmentation

    # Συνάρτηση που επιστρέφει το συνολικό αριθμό δειγμάτων στο dataset
    def __len__(self):
        return len(self.sequences)

    # Επιστρέφει ένα δείγμα (παράθυρο) και την ετικέτα του με βάση τον δείκτη (idx)
    def __getitem__(self, idx):
        x = self.sequences[idx] # Επιλογή της ακολουθίας (παραθύρου)
        y = self.labels[idx] # Επιλογή της αντίστοιχης ετικέτας
        if self.augment:
            # Προσθήκη τυχαίου θορύβου (data augmentation)
            noise = np.random.normal(0, 0.01, x.shape)
            x = x + noise
        x = x.T  # Transpose για να ταιριάζει με την είσοδο του Conv1d (κανάλια, χρόνος)

        return torch.FloatTensor(x), torch.LongTensor([y])

#Ορισμός της αρχιτεκτονικής του Convolutional Neural Network (CNN).

In [10]:
# === 3. CNN Μοντέλο ===
# Ορισμός της αρχιτεκτονικής του Convolutional Neural Network (CNN).
class SafeDrivingCNN(nn.Module):
    # Ο κατασκευαστής του μοντέλου
    def __init__(self, sequence_length=5000, num_classes=2, dropout=0.5):
        super().__init__() # Κλήση του κατασκευαστή της κλάσης (nn.Module)
        # Ορισμός των convolutional επιπέδων
        self.conv = nn.Sequential(
            # Πρώτο Conv1d επίπεδο: 3 κανάλια εισόδου, 32 κανάλια εξόδου, μέγεθος kernel 7
            nn.Conv1d(3, 32, 7, padding=3), nn.BatchNorm1d(32), nn.ReLU(), nn.MaxPool1d(2),
            # Δεύτερο Conv1d επίπεδο: 32 κανάλια εισόδου, 64 κανάλια εξόδου, μέγεθος kernel 5
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            # Τρίτο Conv1d επίπεδο: 64 κανάλια εισόδου, 128 κανάλια εξόδου, μέγεθος kernel 3
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2)
        )
        # Υπολογισμός του μεγέθους της εξόδου μετά τα convolutional και pooling επίπεδα
        conv_out = 128 * (sequence_length // 8) # Το //8 προκύπτει από τα τρία MaxPool1d(2)

        # Ορισμός των fully connected (πλήρως συνδεδεμένων) επιπέδων
        self.fc = nn.Sequential(
            # Πρώτο γραμμικό επίπεδο
            nn.Linear(conv_out, 256), nn.ReLU(), nn.Dropout(dropout),
            # Δεύτερο γραμμικό επίπεδο
            nn.Linear(256, 64), nn.ReLU(), nn.Dropout(dropout),
            # Τρίτο γραμμικό επίπεδο (επίπεδο εξόδου)
            nn.Linear(64, num_classes)
        )

    # Συνάρτηση forward
    def forward(self, x):
        x = self.conv(x) # Πέρασμα από τα convolutional επίπεδα
        x = x.view(x.size(0), -1) # "Ίσιωμα" της εξόδου για τα fully connected επίπεδα (Flatten)
        return self.fc(x) # Πέρασμα από τα fully connected επίπεδα και επιστροφή της εξόδου

#Εκπαίδευση με Grid Search

In [11]:
# === 4. Εκπαίδευση με Grid Search ===
# Eύρεση του καλύτερου συνδυασμού υπερπαραμέτρων.
def grid_search_train(csv_file, label_file, window_size=5000, batch_size=8, epochs=5, use_augmentation=True):
    # Δημιουργία αντικειμένου επεξεργασίας δεδομένων
    processor = DrivingDataProcessor(csv_file)
    # Δημιουργία ακολουθιών και ετικετών από τα αρχεία
    sequences, labels = processor.create_sequences_with_labels(window_size=window_size, label_file=label_file)

    # Διαχωρισμός των δεδομένων σε training και testing sets
    # stratify=labels εξασφαλίζει ότι η αναλογία των κλάσεων είναι ίδια σε train και test με 20% για το test set
    train_X, test_X, train_y, test_y = train_test_split(
        sequences, labels, test_size=0.2, stratify=labels, random_state=42) # random_state για αναπαραγωγιμότητα

    # Δημιουργώ PyTorch Dataset και DataLoader
    train_set = SlidingWindowDataset(train_X, train_y, augment=use_augmentation)
    test_set = SlidingWindowDataset(test_X, test_y, augment=False) # Δεν κάνουμε augmentation στο test set
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True) # shuffle=True για τυχαία σειρά στα training batches
    test_loader = DataLoader(test_set, batch_size=batch_size) # Δεν χρειάζεται shuffle στο test loader

    # Έλεγχος διαθέσιμης συσκευής (GPU ή CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nΧρήση συσκευής: {device}")
    print(f"Χρήση Data Augmentation: {use_augmentation}")

    # Ορισμός των υπερπαραμέτρων που θα δοκιμάσουμε στο Grid Search
    learning_rates = [1e-3, 1e-4]
    weight_decays = [0, 1e-5]
    dropouts = [0.3, 0.5]
    # Δημιουργία όλων των πιθανών συνδυασμών των υπερπαραμέτρων
    param_grid = list(product(learning_rates, weight_decays, dropouts))

    # Αρχικοποίηση μεταβλητών για την παρακολούθηση του καλύτερου μοντέλου
    best_acc = 0
    best_model = None
    best_config = None
    best_true_labels = []
    best_predictions = []

    # Επανάληψη για κάθε συνδυασμό υπερπαραμέτρων
    for i, (lr, wd, do) in enumerate(param_grid):
        print(f"\nΠείραμα {i+1}/{len(param_grid)} | LR={lr}, WD={wd}, Dropout={do}")
        # Δημιουργία νέου μοντέλου για τον τρέχοντα συνδυασμό παραμέτρων
        model = SafeDrivingCNN(sequence_length=window_size, num_classes=len(np.unique(labels)), dropout=do).to(device)
        # Ορισμός του Optimizer (Adam)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        # Ορισμός της συνάρτησης απώλειας (Cross-Entropy Loss)
        criterion = nn.CrossEntropyLoss()

        # Εκπαίδευση του μοντέλου για συγκεκριμένο αριθμό epochs
        for epoch in range(epochs):
            model.train() # Θέση του μοντέλου σε training mode
            total_loss, correct, total = 0, 0, 0
            # Επανάληψη σε batches από τα training data
            for data, targets in train_loader:
                data, targets = data.to(device), targets.squeeze().to(device) # Μεταφορά δεδομένων στη συσκευή (GPU/CPU)
                optimizer.zero_grad() # Μηδενισμός των gradients
                outputs = model(data) # Πρόβλεψη του μοντέλου (forward pass)
                loss = criterion(outputs, targets) # Υπολογισμός της απώλειας
                loss.backward() # Υπολογισμός των gradients (backward pass)
                optimizer.step() # Ενημέρωση των βαρών του μοντέλου
                total_loss += loss.item() # Προσθήκη της απώλειας του batch στη συνολική απώλεια
                preds = outputs.argmax(1) # Λήψη της κλάσης με τη μεγαλύτερη πιθανότητα
                correct += (preds == targets).sum().item() # Μέτρηση σωστών προβλέψεων
                total += targets.size(0) # Μέτρηση συνολικών δειγμάτων στο batch
            acc = correct / total # Υπολογισμός ακρίβειας στο training set
            print(f"Epoch {epoch+1}: Loss={total_loss:.4f} | Accuracy={acc:.2%}")

        # Αξιολόγηση του μοντέλου στο test set μετά από κάθε epoch
        model.eval() # Θέση του μοντέλου σε evaluation mode
        correct, total = 0, 0
        current_true_labels = [] # Λίστα για τις πραγματικές ετικέτες του test set
        current_predictions = [] # Λίστα για τις προβλέψεις του μοντέλου στο test set
        with torch.no_grad(): # Απενεργοποίηση υπολογισμού gradients κατά την αξιολόγηση
            # Επανάληψη σε batches από τα testing data
            for data, targets in test_loader:
                data, targets = data.to(device), targets.squeeze().to(device) # Μεταφορά δεδομένων στη συσκευή
                preds = model(data).argmax(1) # Πρόβλεψη και λήψη της κλάσης
                correct += (preds == targets).sum().item() # Μέτρηση σωστών προβλέψεων
                total += targets.size(0) # Μέτρηση συνολικών δειγμάτων
                current_true_labels.extend(targets.cpu().numpy()) # Αποθήκευση πραγματικών ετικετών
                current_predictions.extend(preds.cpu().numpy()) # Αποθήκευση προβλέψεων

        acc = correct / total # Υπολογισμός ακρίβειας στο test set
        print(f"Accuracy στο test set: {acc:.2%}")

        # Ποιό μοντέλο είναι το καλύτερο?
        if acc > best_acc:
            best_acc = acc # Καλύτερη ακρίβεια
            best_model = model # Αποθήκευση του καλύτερου μοντέλου
            best_config = (lr, wd, do) # Αποθήκευση της καλύτερης ρύθμισης υπερπαραμέτρων
            best_true_labels = current_true_labels # Αποθήκευση των πραγματικών ετικετών για το καλύτερο μοντέλο
            best_predictions = current_predictions # Αποθήκευση των προβλέψεων για το καλύτερο μοντέλο

    # Εκτύπωση των αποτελεσμάτων του Grid Search
    print(f"\nΚαλύτερο config: LR={best_config[0]}, WD={best_config[1]}, Dropout={best_config[2]}")
    print(f"Best Test Accuracy: {best_acc:.2%}\n")

    # Αναφορά ταξινόμησης (Classification Report) για το καλύτερο μοντέλο
    print("Classification Report (Test Set):")
    # Μετατροπή των ονομάτων των κλάσεων σε strings για την αναφορά
    target_names = [str(c) for c in processor.label_encoder.classes_]
    print(classification_report(best_true_labels, best_predictions, target_names=target_names))

    # Confusion Matrix μόνο για το καλύτερο μοντέλο
    cm = confusion_matrix(best_true_labels, best_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names, yticklabels=target_names)
    plt.title('Confusion Matrix - Test Set (Best Model)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig('confusion_matrix_test_set.png', dpi=300)
    plt.close()
    print("Confusion Matrix αποθηκεύτηκε ως confusion_matrix_test_set.png")

    # Επιστροφή του καλύτερου μοντέλου και του processor
    return best_model, processor


#Πρόβλεψη σε hold-out δεδομένα (10%) μη χρησιμοποιημένο

In [12]:
# === 5. Πρόβλεψη σε hold-out δεδομένα ===
# Πρόβλεψη σε ένα ξεχωριστό set δεδομένων (hold-out).
# 10% των συνολικών αρχικών δεδομένων) είναι ένα ξεχωριστό σύνολο δεδομένων που δε χρησιμοποιήθηκε καθόλου κατά τη διάρκεια
# του Grid Search (ούτε για training ούτε για testing)
def predict_with_sliding_windows(model, processor, new_data, window_size=5000):
    model.eval() # Θέση του μοντέλου σε evaluation mode
    device = next(model.parameters()).device # Εύρεση της συσκευής που χρησιμοποιεί το μοντέλο
    # Επιλογή των στηλών επιτάχυνσης από τα νέα δεδομένα
    acc_data = new_data[['Acceleration_X_Lateral','Acceleration_Y_Longitudinal','Acceleration_Z_Vertical']].values
    predictions = [] # Λίστα για να αποθηκεύσουμε τις πιθανότητες πρόβλεψης για κάθε παράθυρο

    for start in range(0, len(acc_data) - window_size + 1, window_size):
        window = acc_data[start:start + window_size]
        tensor = torch.FloatTensor(window.T).unsqueeze(0).to(device) # .T για σωστή διάσταση, .unsqueeze(0) για batch dimension
        with torch.no_grad(): # Απενεργοποίηση υπολογισμού gradients
            output = model(tensor)
            # Υπολογισμός πιθανοτήτων με softmax
            prob = torch.softmax(output, dim=1).cpu().numpy()[0]
            predictions.append(prob) # Αποθήκευση των πιθανοτήτων του παραθύρου

    # Υπολογισμός της μέσης πιθανότητας για όλες τις κλάσες σε όλα τα παράθυρα
    avg_probs = np.mean(predictions, axis=0)
    # Επιλογή της κλάσης με τη μεγαλύτερη μέση πιθανότητα ως τελική πρόβλεψη
    predicted_class = int(np.argmax(avg_probs))
    # Αποκωδικοποίηση της κλάσης στην αρχική της μορφή (π.χ. 0 -> 'safe')
    decoded_label = processor.label_encoder.inverse_transform([predicted_class])[0]
    # Mέγιστη μέση πιθανότητα ως εμπιστοσύνη στην πρόβλεψη
    confidence = float(np.max(avg_probs))

    # Εκτύπωση πρόβλεψης
    print(f"\nΤελική πρόβλεψη: {decoded_label} με εμπιστοσύνη {confidence:.2%}")
    # Επιστροφή της προβλεπόμενης κλάσης, της αποκωδικοποιημένης ετικέτας, της εμπιστοσύνης και των πιθανοτήτων ανά παράθυρο
    return predicted_class, decoded_label, confidence, predictions

#Αυστηρή απόφαση κινδύνου, ενα παράθυρο risky ισοδυναμεί με όλη η διαδρομή risky

In [13]:
# === 6. Αυστηρή απόφαση κινδύνου ===
# Συνάρτηση που λαμβάνει μια πιο "αυστηρή" απόφαση για όλη τη διαδρομή
# με βάση τις προβλέψεις ανά παράθυρο.
def strict_risk_decision(predictions, processor, risk_threshold=0.5):
    # Μετράμε πόσα παράθυρα χαρακτηρίστηκαν ως 'risky' (κλάση 1) με πιθανότητα > risk_threshold
    risky_windows = sum(1 for p in predictions if p[1] > risk_threshold)
    total_windows = len(predictions) # Συνολικός αριθμός παραθύρων

    # Αν δεν υπάρχουν παράθυρα, επιστρέφουμε 'safe' με 0 εμπιστοσύνη
    if total_windows == 0:
        return 0, processor.label_encoder.inverse_transform([0])[0], 0.0

    # Υπολογισμός της αναλογίας των 'risky' παραθύρων
    risky_ratio = risky_windows / total_windows

    # Αν υπάρχει έστω και ένα 'risky' παράθυρο, θεωρούμε όλη τη διαδρομή 'risky'
    if risky_ratio > 0:
        predicted_class = 1 # Η κλάση είναι 1 (risky)
        confidence = risky_ratio # Η εμπιστοσύνη είναι η αναλογία των risky παραθύρων
    else:
        # Αν δεν υπάρχει κανένα 'risky' παράθυρο, θεωρούμε τη διαδρομή 'safe'
        predicted_class = 0 # Η κλάση είναι 0 (safe)
        confidence = 1.0 # Η εμπιστοσύνη στη 'safe' κλάση είναι υψηλή (100%)

    # Αποκωδικοποίηση της τελικής κλάσης
    decoded_label = processor.label_encoder.inverse_transform([predicted_class])[0]
    # Επιστροφή της τελικής κλάσης, της ετικέτας και της εμπιστοσύνης
    return predicted_class, decoded_label, confidence

#Εκτέλεση του κυρίως προγράμματος

In [14]:
# === 7. Εκτέλεση του κυρίως προγράμματος ===
if __name__ == "__main__":
    # Ορισμός των αρχείων και των βασικών παραμέτρων
    csv_file = "acceleration_data.csv" # Αρχείο με δεδομένα επιτάχυνσης
    label_file = "empirical_labels.csv" # Αρχείο με ετικέτες (0 ή 1)
    window_size = 5000 # Μέγεθος του συρόμενου παραθύρου (σε δείγματα)
    batch_size = 8 # Μέγεθος batch για την εκπαίδευση
    epochs = 5 # Αριθμός epochs για την εκπαίδευση κάθε μοντέλου στο grid search

    # === Εκπαίδευση με Grid Search ===
    # Καλούμε τη συνάρτηση grid_search_train για να εκπαιδεύσουμε το μοντέλο με Grid Search
    # και να βρούμε τις καλύτερες υπερπαραμέτρους. use_augmentation είναι False σε αυτή την περίπτωση.
    model, processor = grid_search_train(
        csv_file=csv_file,
        label_file=label_file,
        window_size=window_size,
        batch_size=batch_size,
        epochs=epochs,
        use_augmentation=True # Δεν χρησιμοποιούμε data augmentation σε αυτό το run-αλλάζει την default επιλογή
    )

    # === Δημιουργία hold-out set ===
    # Ξαναδημιουργούμε όλες τις ακολουθίες και ετικέτες για να πάρουμε ένα ξεχωριστό hold-out set
    # που δεν χρησιμοποιήθηκε καθόλου στην εκπαίδευση ή την αξιολόγηση του grid search.
    all_sequences, all_labels = processor.create_sequences_with_labels(window_size, label_file)
    total_windows = len(all_sequences) # Συνολικός αριθμός παραθύρων στα αρχικά δεδομένα
    # Διαχωρισμός για τη δημιουργία του hold-out set (10% των συνολικών παραθύρων)
    _, test_X, _, test_y = train_test_split(
        all_sequences, all_labels, test_size=0.1, stratify=all_labels, random_state=42
    )

    # === Εκτυπώσεις ===
    holdout_windows = len(test_X)
    print(f"\nΣυνολικά παράθυρα: {total_windows}")
    print(f"Παράθυρα στο hold-out set: {holdout_windows}")
    print(f"Χρονική διάρκεια κάθε παραθύρου: {window_size} δείγματα")
    print(f"Συνολική διάρκεια hold-out: {holdout_windows * window_size} δείγματα")

    # === Προετοιμασία hold-out dataframe ===
    df_holdout = pd.DataFrame(np.concatenate(test_X), columns=[
        "Acceleration_X_Lateral", "Acceleration_Y_Longitudinal", "Acceleration_Z_Vertical"
    ])

    # === Πρόβλεψη στο hold-out ===
    # Καλούμε τη συνάρτηση predict_with_sliding_windows για να κάνουμε προβλέψεις
    # στο hold-out set χρησιμοποιώντας το καλύτερο μοντέλο από το grid search.
    predicted_class, decoded_label, confidence, predictions = predict_with_sliding_windows(
        model, processor, df_holdout, window_size=window_size
    )

    # === Αυστηρή απόφαση ===
    # Καλούμε τη συνάρτηση strict_risk_decision για να πάρουμε τη "αυστηρή" τελική απόφαση
    # για όλη τη διαδρομή του hold-out set.
    strict_class, strict_label, strict_conf = strict_risk_decision(predictions, processor)
    print(f"\nΑυστηρή απόφαση: {strict_label} με εμπιστοσύνη {strict_conf:.2%}")

    # === Γράφημα πιθανότητας 'risky' ανά παράθυρο ===
    # Δημιουργία γραφήματος που δείχνει την πιθανότητα η κάθε παράθυρο να είναι 'risky'
    risky_probs = [p[1] for p in predictions] # Λήψη των πιθανοτήτων για την κλάση 'risky' (κλάση 1)
    plt.figure(figsize=(10, 4))
    plt.plot(range(len(risky_probs)), risky_probs, marker='o', color='blue', label='P(risky)')
    plt.axhline(0.5, color='gray', linestyle='--', label='Κατώφλι 50%') # Οριζόντια γραμμή στο 50%
    plt.title("Πιθανότητα 'risky' ανά παράθυρο (Hold-out)")
    plt.xlabel("Αριθμός παραθύρου")
    plt.ylabel("P(risky)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig("risky_per_window_holdout.png", dpi=300) # Αποθήκευση του γραφήματος
    plt.close() # Κλείσιμο της φιγούρας
    print("Αποθηκεύτηκε ως risky_per_window_holdout.png")

    # === Μέση πιθανότητα 'risky' ===
    # Δημιουργία γραφήματος που δείχνει τη μέση πιθανότητα να είναι 'risky' για όλο το hold-out set
    mean_risky = np.mean(risky_probs) # Υπολογισμός της μέσης πιθανότητας 'risky'
    plt.figure(figsize=(6, 4))
    # Δημιουργία μπάρας, χρώμα κόκκινο αν η μέση πιθανότητα > 0.5, πράσινο αλλιώς
    plt.bar(["Μέση πιθανότητα 'risky'"], [mean_risky], color='red' if mean_risky > 0.5 else 'green')
    plt.axhline(0.5, color='gray', linestyle='--', label='Κατώφλι 50%') # Οριζόντια γραμμή στο 50%
    plt.ylim(0, 1) # Ορισμός ορίων στον άξονα y από 0 έως 1
    plt.title("Μέση εκτίμηση διαδρομής (Hold-out Set)")
    plt.ylabel("Πιθανότητα 'risky'")
    plt.legend() # Εμφάνιση λεζάντας
    plt.tight_layout() # Προσαρμογή διάταξης
    plt.savefig("inference_mean_risk.png", dpi=300) # Αποθήκευση του γραφήματος
    plt.close() # Κλείσιμο της φιγούρας
    print("Η μέση εκτίμηση αποθηκεύτηκε ως inference_mean_risk.png")


Χρήση συσκευής: cpu
Χρήση Data Augmentation: True

Πείραμα 1/8 | LR=0.001, WD=0, Dropout=0.3
Epoch 1: Loss=40.1348 | Accuracy=73.04%
Epoch 2: Loss=31.2174 | Accuracy=83.48%
Epoch 3: Loss=32.7722 | Accuracy=72.17%
Epoch 4: Loss=7.2870 | Accuracy=82.61%
Epoch 5: Loss=7.6114 | Accuracy=75.65%
Accuracy στο test set: 89.66%

Πείραμα 2/8 | LR=0.001, WD=0, Dropout=0.5
Epoch 1: Loss=96.5982 | Accuracy=67.83%
Epoch 2: Loss=48.2134 | Accuracy=81.74%
Epoch 3: Loss=34.2309 | Accuracy=76.52%
Epoch 4: Loss=20.4724 | Accuracy=87.83%
Epoch 5: Loss=23.6794 | Accuracy=81.74%
Accuracy στο test set: 68.97%

Πείραμα 3/8 | LR=0.001, WD=1e-05, Dropout=0.3
Epoch 1: Loss=44.4138 | Accuracy=81.74%
Epoch 2: Loss=55.0687 | Accuracy=72.17%
Epoch 3: Loss=17.5673 | Accuracy=73.91%
Epoch 4: Loss=13.0256 | Accuracy=83.48%
Epoch 5: Loss=4.0325 | Accuracy=87.83%
Accuracy στο test set: 89.66%

Πείραμα 4/8 | LR=0.001, WD=1e-05, Dropout=0.5
Epoch 1: Loss=86.8945 | Accuracy=76.52%
Epoch 2: Loss=65.7482 | Accuracy=77.39%
Ep